In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import selenium.webdriver.support.ui as ui
import selenium.webdriver.support.expected_conditions as EC
import os, time,datetime
import pandas as pd
import numpy as np

team_dictionary={
'Atlanta Hawks':'ATL',
'Brooklyn Nets':'BRK',
'Boston Celtics':'BOS',
'Charlotte Hornets':'CHO',
'Chicago Bulls':'CHI',
'Cleveland Cavaliers':'CLE',
'Dallas Mavericks':'DAL',
'Denver Nuggets':'DEN',
'Detroit Pistons':'DET',
'Golden State Warriors':'GSW',
'Houston Rockets':'HOU',
'Indiana Pacers':'IND',
'Los Angeles Clippers':'LAC',
'Los Angeles Lakers':'LAL',
'Memphis Grizzlies':'MEM',
'Miami Heat':'MIA',
'Milwaukee Bucks':'MIL',
'Minnesota Timberwolves':'MIN',
'New Orleans Pelicans':'NOP',
'New York Knicks':'NYK',
'Oklahoma City Thunder':'OKC',
'Orlando Magic':'ORL',
'Philadelphia 76ers':'PHI',
'Phoenix Suns':'PHO',
'Portland Trail Blazers':'POR',
'Sacramento Kings':'SAC',
'San Antonio Spurs':'SAS',
'Toronto Raptors':'TOR',
'Utah Jazz':'UTA',
'Washington Wizards':'WAS'
}

def scores_to_list(score_splits,unique_id):
    #len either 22/25/28
    score_splits=score_splits.text.split()
    if len(score_splits)==22:
        away=score_splits[10:15]
        home=score_splits[16:21]
    elif len(score_splits)==25:
        away=score_splits[11:17]
        home=score_splits[18:24]
    elif len(score_splits)==28:
        away=score_splits[12:19]
        home=score_splits[20:27]
    elif len(score_splits)==31:
        away=score_splits[13:21]
        home=score_splits[22:30]
    else:
        print("quad OT???")
    combined=away[:5] + home[:5]
    xfer_to_df=[unique_id,combined[5],combined[0],combined[6],combined[7],combined[8],combined[9],combined[1],combined[2],combined[3],combined[4]]
    return xfer_to_df

def win_tracker(team_normal,team_altered,Other_normal,Other_altered):
    team_normal=int(team_normal)
    team_altered=int(team_altered)
    Other_normal=int(Other_normal)
    Other_altered=int(Other_altered)
    if team_normal>Other_normal and team_altered>Other_altered:
        result="WW"
    elif team_normal>Other_normal and team_altered==Other_altered:
        result="WT"
    elif team_normal>Other_normal and team_altered<Other_altered:
        result="WL"
    elif team_normal<Other_normal and team_altered>Other_altered:
        result="LW"
    elif team_normal<Other_normal and team_altered==Other_altered:
        result="LT"
    elif team_normal<Other_normal and team_altered<Other_altered:
        result="LL"
    elif team_normal==Other_normal and team_altered>Other_altered:
        result="TW"
    elif team_normal==Other_normal and team_altered==Other_altered:
        result="TT"
    elif team_normal==Other_normal and team_altered<Other_altered:
        result="TL"    
    return result

def win_tracker_short(team,other):
    team=int(team)
    other=int(other)
    if team>other:
        result="W"
    elif team<other:
        result="L"
    elif team==other:
        result="T"  
    return result

In [58]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

def save_progress(df,iteration):
    writer = pd.ExcelWriter('A:\\Programming files\\Python scripts\\Data_Analysis\\nba_third_quarter{}.xlsx'.format(iteration), engine='xlsxwriter')
    df.to_excel(writer, index=False, sheet_name='full_df')
    writer.save()
    
def season_wo_third(df,team):
    season=[]
    #filter for cases where the team is playing
    g_wo_third=df.loc[(df['Home_Team'] == team) | (df['Away_Team'] == team)]
    g_wo_third=g_wo_third.reset_index(drop=True)
    for i in range(len(g_wo_third)):
        #get the score of the team as if the 3rd quarter wasn't played
        if g_wo_third.loc[i,"Home_Team"]==team:
            team_normal=int(g_wo_third.loc[i,"Home_Total"])
            Other_normal=int(g_wo_third.loc[i,"Away_Total"])
            team_altered=int(g_wo_third.loc[i,"Home_wo_3"])
            Other_altered=int(g_wo_third.loc[i,"Away_wo_3"])
        else:
            team_normal=int(g_wo_third.loc[i,"Away_Total"])
            Other_normal=int(g_wo_third.loc[i,"Home_Total"])
            team_altered=int(g_wo_third.loc[i,"Away_wo_3"])
            Other_altered=int(g_wo_third.loc[i,"Home_wo_3"])
        #print("season_wo_third: {}-{}-{}-{}-{}".format(team,team_normal,team_altered,Other_normal,Other_altered))
        season.append(win_tracker(team_normal,team_altered,Other_normal,Other_altered))
    return season

def half_time_conversion(df,team):
    season=[]
    #filter for cases where the team is playing
    full_game=df.loc[(df['Home_Team'] == team) | (df['Away_Team'] == team)]
    full_game=full_game.reset_index(drop=True)
    for i in range(len(full_game)):
        if full_game.loc[i,"Home_Team"]==team:
            team_normal=int(full_game.loc[i,"Home_Total"])
            Other_normal=int(full_game.loc[i,"Away_Total"])
            team_altered=int(full_game.loc[i,"Home_Q1"]+full_game.loc[i,"Home_Q2"])
            Other_altered=int(full_game.loc[i,"Away_Q1"]+full_game.loc[i,"Away_Q2"])
        else:
            team_normal=int(full_game.loc[i,"Away_Total"])
            Other_normal=int(full_game.loc[i,"Home_Total"])        	
            team_altered=int(full_game.loc[i,"Away_Q1"]+full_game.loc[i,"Away_Q2"])
            Other_altered=int(full_game.loc[i,"Home_Q1"]+full_game.loc[i,"Home_Q2"])
        #print("half time conversion: {}-{}-{}-{}-{}".format(team,team_normal,team_altered,Other_normal,Other_altered))
        season.append(win_tracker(team_normal,team_altered,Other_normal,Other_altered))
    return season

#trying to figure out the point differential in the 3rd quarter
def third_performance(team):
    #filter for cases where the team is playing
    third_q_df=df.loc[(df['Home_Team'] == team) | (df['Away_Team'] == team)]
    third_q_df=third_q_df.reset_index(drop=True)
    season=[]
    for i in range(len(third_q_df)):
        #Find out how much the team vs other team scored in the 3rd quarter
        if third_q_df.loc[i,"Home_Team"]==team:
            team=int(third_q_df.loc[i,'Home_Q3'])
            other=int(third_q_df.loc[i,'Away_Q3'])
        else:
            team=int(third_q_df.loc[i,'Away_Q3'])
            other=int(third_q_df.loc[i,'Home_Q3'])
        #for each game, take the differential and append it    
        season.append(team-other)
    return season


def third_performance_solo(df,team):
    #filter for cases where the team is playing
    third_q_df=df.loc[(df['Home_Team'] == team) | (df['Away_Team'] == team)]
    third_q_df=third_q_df.reset_index(drop=True)
    season=[]
    for i in range(len(third_q_df)):
        #Find out how much the team vs other team scored in the 3rd quarter
        if third_q_df.loc[i,"Home_Team"]==team:
            team=int(third_q_df.loc[i,'Home_Q3'])
            other=int(third_q_df.loc[i,'Away_Q3'])
        else:
            team=int(third_q_df.loc[i,'Away_Q3'])
            other=int(third_q_df.loc[i,'Home_Q3'])
        #for each game, take the differential and append it    
        season.append(win_tracker_short(team,other))
    return season

def third_performance_net(team):
    #filter for cases where the team is playing
    third_q_df=df.loc[(df['Home_Team'] == team) | (df['Away_Team'] == team)]
    third_q_df=third_q_df.reset_index(drop=True)
    third_quarter_avg=[]
    for i in range(len(third_q_df)):
        #Find out how much the team vs other team scored in the 3rd quarter
        if third_q_df.loc[i,"Home_Team"]==team:
            team=int(third_q_df.loc[i,'Home_Q3'])
            #other=int(third_q_df.loc[i,'Away_Q3'])
        else:
            team=int(third_q_df.loc[i,'Away_Q3'])
            #other=int(third_q_df.loc[i,'Home_Q3'])
        #for each game, take the differential and append it    
        third_quarter_avg.append(team)
    third_quarter_avg=sum(third_quarter_avg)/82
    return third_quarter_avg

def third_ratio(team):
    #filter for cases where the team is playing
    third_q_df=df.loc[(df['Home_Team'] == team) | (df['Away_Team'] == team)]
    third_q_df=third_q_df.reset_index(drop=True)
    third_ratio=[]
    for i in range(len(third_q_df)):
        #Find out how much the team vs other team scored in the 3rd quarter
        if third_q_df.loc[i,"Home_Team"]==team:
            team=int(third_q_df.loc[i,'Home_Q3'])
            other=int(third_q_df.loc[i,'Away_Q3'])
        else:
            team=int(third_q_df.loc[i,'Away_Q3'])
            other=int(third_q_df.loc[i,'Home_Q3'])
        #for each game, take the differential and append it    
        third_ratio.append(team/other)
    third_ratio=mean(third_ratio)
    return third_ratio


In [42]:
def scrape_to_save(games_list,iteration):
    list_of_game_scores=[]
    broken_list=[]

    for i in games_list:
        print("Attempting with {} at {}".format(i,datetime.datetime.now()))
        url="https://www.basketball-reference.com/boxscores/{}.html".format(i)
        location="A:\Programming files\Python scripts\dbbackups\phantomjs.exe"
        driver = webdriver.PhantomJS(location)
        driver.set_window_size(1000,1000)

        driver.get(url)
        try:
            ui.WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.ID, "line_score")))
            line_score_table = driver.find_element_by_xpath("//div[@id='all_line_score']")
            list_of_game_scores.append(scores_to_list(line_score_table,i))
        except Exception as e:
            driver.save_screenshot('screenshotsel.png')
            broken_list.append(i)
        #print(line_score_table.text)
        #print(scores_to_list(line_score_table))
        driver.close()
        driver.quit()

    print("\n\nFinished set")

    df=pd.DataFrame()
    df=df.append(list_of_game_scores[0],ignore_index=True)
    #print(type(df))
    df=df.T
    df=df.rename({0:'game_id',1:'Home_Team',2:'Away_Team',3:'Home_Q1',4:'Home_Q2',5:'Home_Q3',6:'Home_Q4',7:'Away_Q1',8:'Away_Q2',9:'Away_Q3',10:'Away_Q4'}, axis ='columns')
 
 
    for game in list_of_game_scores[1:]:
        newdf=pd.DataFrame()
        newdf=newdf.append(game,ignore_index=True).T
        newdf=newdf.rename({0:'game_id',1:'Home_Team',2:'Away_Team',3:'Home_Q1',4:'Home_Q2',5:'Home_Q3',6:'Home_Q4',7:'Away_Q1',8:'Away_Q2',9:'Away_Q3',10:'Away_Q4'}, axis ='columns')
        df=df.append(newdf)
    save_progress(df,"iteration{}".format(iteration))
    print(broken_list)
    return df

def win_loss(season):
    RW=0
    RL=0
    RT=0
    SW=0
    SL=0
    ST=0
    for i in season:
        if i == "WW":
            RW+=1
            SW+=1
        elif i == "WT":
            RW+=1
            ST+=1
        elif i == "WL":
            RW+=1
            SL+=1
        elif i == "LW":
            RL+=1
            SW+=1
        elif i == "LT":
            RL+=1
            ST+=1
        elif i == "LL":
            RL+=1
            SL+=1
        elif i == "TW":
            RT+=1
            SW+=1
        elif i == "TT":
            RT+=1
            ST+=1
        elif i == "TL":
            RT+=1
            SL+=1
    real_season="{}-{}-{}".format(RW,RL,RT)
    sim_season="{}-{}-{}".format(SW,SL,ST)
    return real_season,sim_season

def win_loss_short(season):
    W=0
    L=0
    T=0
    for i in season:
        if i == "W":
            W+=1
        elif i == "T":
            T+=1
        elif i == "L":
            L+=1
    sim_season="{}-{}-{}".format(W,L,T)
    return sim_season

In [ ]:
full_list=['201710170CLE','201710170GSW','201710180BOS','201710180DAL','201710180DET','201710180IND','201710180MEM','201710180ORL','201710180PHO','201710180SAC','201710180SAS','201710180UTA','201710180WAS','201710190LAL','201710190OKC','201710190TOR','201710200BRK','201710200CHO','201710200DAL','201710200IND','201710200MIL','201710200MIN','201710200NOP','201710200PHI','201710200PHO','201710200WAS','201710210CHI','201710210CLE','201710210DEN','201710210HOU','201710210LAC','201710210MEM','201710210MIA','201710210MIL','201710210NYK','201710210TOR','201710210UTA','201710220BRK','201710220LAL','201710220OKC','201710230DAL','201710230DEN','201710230DET','201710230HOU','201710230MIA','201710230MIL','201710230PHO','201710230SAS','201710240BOS','201710240CLE','201710240LAC','201710240MIN','201710240ORL','201710240POR','201710250BRK','201710250CHO','201710250DAL','201710250DET','201710250GSW','201710250LAL','201710250MIA','201710250OKC','201710250PHI','201710250PHO','201710260CHI','201710260MEM','201710260MIL','201710260POR','201710260SAC','201710270ATL','201710270CHO','201710270GSW','201710270LAL','201710270MIN','201710270NYK','201710270ORL','201710280CHI','201710280DAL','201710280LAC','201710280MEM','201710280MIA','201710280NOP','201710280POR','201710280UTA','201710290ATL','201710290BRK','201710290CHO','201710290CLE','201710290GSW','201710290IND','201710290SAC','201710300BOS','201710300HOU','201710300LAC','201710300MEM','201710300MIA','201710300NOP','201710300NYK','201710300POR','201710300UTA','201710310BRK','201710310IND','201710310LAL','201710310MIL','201711010BOS','201711010CHO','201711010CLE','201711010DEN','201711010LAC','201711010MEM','201711010MIA','201711010NOP','201711010NYK','201711010PHI','201711010UTA','201711010WAS','201711020POR','201711020SAS','201711030ATL','201711030DAL','201711030DEN','201711030DET','201711030LAL','201711030NYK','201711030OKC','201711030ORL','201711030PHI','201711030SAS','201711030UTA','201711030WAS','201711040CHI','201711040DEN','201711040DET','201711040LAC','201711040MIN','201711050CLE','201711050HOU','201711050LAC','201711050LAL','201711050MIN','201711050NYK','201711050ORL','201711050POR','201711050SAS','201711050TOR','201711060ATL','201711060GSW','201711060PHO','201711070CLE','201711070DEN','201711070IND','201711070NYK','201711070POR','201711070SAC','201711070SAS','201711070TOR','201711070UTA','201711070WAS','201711080BOS','201711080DET','201711080GSW','201711080ORL','201711080PHO','201711090DEN','201711090HOU','201711090SAC','201711090TOR','201711090WAS','201711100BOS','201711100CHI','201711100DET','201711100OKC','201711100PHO','201711100POR','201711100SAS','201711100UTA','201711110DAL','201711110DEN','201711110GSW','201711110HOU','201711110MIL','201711110NOP','201711110NYK','201711110PHO','201711110SAS','201711110UTA','201711110WAS','201711120BOS','201711120DET','201711120IND','201711120OKC','201711130GSW','201711130LAC','201711130MIL','201711130NOP','201711130NYK','201711130PHO','201711130POR','201711130UTA','201711130WAS','201711140BRK','201711140DAL','201711140HOU','201711150ATL','201711150CHO','201711150LAL','201711150MEM','201711150MIA','201711150MIL','201711150MIN','201711150NOP','201711150NYK','201711150OKC','201711150POR','201711160BOS','201711160PHO','201711170BRK','201711170CHI','201711170CLE','201711170DAL','201711170DEN','201711170IND','201711170LAL','201711170SAC','201711170SAS','201711170TOR','201711170WAS','201711180ATL','201711180CHO','201711180DAL','201711180MEM','201711180ORL','201711180PHI','201711180POR','201711190BRK','201711190LAL','201711190MIA','201711190MIN','201711190PHO','201711190TOR','201711200CHO','201711200DAL','201711200DET','201711200MEM','201711200MIL','201711200NOP','201711200NYK','201711200ORL','201711200PHI','201711200SAC','201711200SAS','201711210LAL','201711220ATL','201711220CHO','201711220CLE','201711220HOU','201711220MEM','201711220MIA','201711220MIN','201711220NOP','201711220NYK','201711220OKC','201711220PHI','201711220PHO','201711220SAC','201711220UTA','201711240ATL','201711240BOS','201711240BRK','201711240CLE','201711240DEN','201711240GSW','201711240IND','201711240MIN','201711240OKC','201711240PHO','201711250ATL','201711250CHO','201711250DAL','201711250GSW','201711250HOU','201711250IND','201711250PHI','201711250SAC','201711250UTA','201711250WAS','201711260CHI','201711260MEM','201711260MIN','201711270BOS','201711270GSW','201711270HOU','201711270IND','201711270LAC','201711270NYK','201711270PHI','201711270SAS','201711280CHI','201711280CLE','201711280MIN','201711280SAC','201711280UTA','201711290DAL','201711290DET','201711290HOU','201711290LAL','201711290NOP','201711290NYK','201711290ORL','201711290PHI','201711290SAS','201711290TOR','201711300ATL','201711300BOS','201711300DEN','201711300LAC','201711300POR','201712010CHI','201712010MEM','201712010MIA','201712010OKC','201712010ORL','201712010TOR','201712010UTA','201712010WAS','201712020BOS','201712020BRK','201712020CLE','201712020DAL','201712020DEN','201712020MIL','201712020PHI','201712020POR','201712030LAL','201712030MIA','201712030MIN','201712030NYK','201712030OKC','201712040ATL','201712040BOS','201712040CHI','201712040CHO','201712040DAL','201712040IND','201712040MEM','201712040NOP','201712040PHI','201712040SAS','201712040UTA','201712050OKC','201712050POR','201712050TOR','201712060BOS','201712060CHO','201712060CLE','201712060IND','201712060LAC','201712060MIL','201712060NOP','201712060NYK','201712060ORL','201712060SAS','201712070BRK','201712070PHI','201712070PHO','201712070UTA','201712080CHO','201712080DET','201712080IND','201712080MEM','201712080MIL','201712080NOP','201712080ORL','201712080SAS','201712090ATL','201712090BRK','201712090CHI','201712090CHO','201712090CLE','201712090LAC','201712090MEM','201712090MIL','201712090PHO','201712090POR','201712100DET','201712100IND','201712100MIN','201712100NOP','201712100NYK','201712100SAC','201712110CHI','201712110GSW','201712110HOU','201712110LAC','201712110MEM','201712110OKC','201712120BRK','201712120CLE','201712120DAL','201712120DET','201712120MIN','201712120NYK','201712120SAC','201712130BOS','201712130CHI','201712130HOU','201712130IND','201712130MIA','201712130NOP','201712130ORL','201712130PHO','201712130WAS','201712140ATL','201712140BRK','201712140CLE','201712140GSW','201712140MIN','201712150BOS','201712150CHO','201712150DEN','201712150HOU','201712150IND','201712150MEM','201712150MIL','201712150ORL','201712150PHI','201712150TOR','201712150WAS','201712160CHO','201712160CLE','201712160HOU','201712160MEM','201712160MIA','201712160MIN','201712160NYK','201712160SAS','201712170BRK','201712170DET','201712170TOR','201712170WAS','201712180ATL','201712180CHI','201712180CHO','201712180DAL','201712180HOU','201712180IND','201712180LAL','201712180MIN','201712180OKC','201712180SAS','201712190MIL','201712190PHI','201712190WAS','201712200ATL','201712200BOS','201712200BRK','201712200CHI','201712200CHO','201712200DAL','201712200DEN','201712200GSW','201712200HOU','201712200LAC','201712200OKC','201712200POR','201712210CLE','201712210NYK','201712210PHI','201712210PHO','201712210UTA','201712220BRK','201712220DET','201712220GSW','201712220HOU','201712220MIA','201712220MIL','201712220OKC','201712220ORL','201712220POR','201712230ATL','201712230BOS','201712230CHO','201712230GSW','201712230IND','201712230LAL','201712230MEM','201712230MIA','201712230PHO','201712230SAC','201712230TOR','201712230UTA','201712230WAS','201712250BOS','201712250GSW','201712250LAL','201712250NYK','201712250OKC','201712260DAL','201712260DEN','201712260DET','201712260LAC','201712260MIA','201712260MIL','201712260PHO','201712260SAS','201712270ATL','201712270CHI','201712270CHO','201712270GSW','201712270IND','201712270LAL','201712270MIN','201712270NOP','201712270OKC','201712270SAC','201712280BOS','201712280MIL','201712280ORL','201712280POR','201712280SAS','201712290CHI','201712290GSW','201712290LAL','201712290MIA','201712290NOP','201712290OKC','201712290SAC','201712290TOR','201712290WAS','201712300ATL','201712300DEN','201712300DET','201712300GSW','201712300NOP','201712300ORL','201712300UTA','201712310BOS','201712310HOU','201712310IND','201712310LAC','201712310OKC','201712310PHO','201712310SAC','201712310WAS','201801010BRK','201801010CHI','201801010MIN','201801010TOR','201801020CLE','201801020LAC','201801020NYK','201801020PHO','201801020SAC','201801030BOS','201801030BRK','201801030CHI','201801030DAL','201801030DEN','201801030LAL','201801030MIA','201801030MIL','201801030ORL','201801030PHI','201801030UTA','201801030WAS','201801040HOU','201801040LAC','201801050BOS','201801050DAL','201801050DEN','201801050LAL','201801050MEM','201801050MIA','201801050MIL','201801050PHI','201801050POR','201801050SAS','201801060BRK','201801060DET','201801060IND','201801060LAC','201801060MIN','201801060ORL','201801060SAC','201801060WAS','201801070DAL','201801070LAL','201801070MIA','201801070PHO','201801070POR','201801080BRK','201801080CHI','201801080GSW','201801080IND','201801080LAC','201801080MIN','201801080NOP','201801080SAC','201801090DAL','201801090LAL','201801090OKC','201801090TOR','201801100BRK','201801100CHO','201801100DEN','201801100GSW','201801100HOU','201801100IND','201801100MEM','201801100MIL','201801100MIN','201801100NYK','201801100WAS','201801110LAL','201801110PHI','201801110SAC','201801110TOR','201801120ATL','201801120CHO','201801120DEN','201801120IND','201801120MIL','201801120MIN','201801120NOP','201801120PHO','201801120WAS','201801130CHI','201801130CHO','201801130DAL','201801130LAC','201801130SAS','201801130TOR','201801130WAS','201801140MIA','201801140MIN','201801140NYK','201801140PHO','201801150ATL','201801150BRK','201801150CHI','201801150CLE','201801150DET','201801150LAC','201801150MEM','201801150OKC','201801150PHI','201801150UTA','201801150WAS','201801160BOS','201801160DEN','201801160ORL','201801160POR','201801170ATL','201801170BRK','201801170CHI','201801170CHO','201801170LAC','201801170MEM','201801170MIL','201801170OKC','201801170SAC','201801170TOR','201801180BOS','201801180CLE','201801180HOU','201801180POR','201801190BRK','201801190DEN','201801190DET','201801190LAL','201801190MEM','201801190TOR','201801190UTA','201801200ATL','201801200CHO','201801200CLE','201801200HOU','201801200MIN','201801200NOP','201801200PHI','201801200POR','201801200UTA','201801210BOS','201801210DET','201801210LAL','201801210SAS','201801220ATL','201801220CHO','201801220DAL','201801220DEN','201801220HOU','201801220LAC','201801220MEM','201801220MIL','201801220NOP','201801230GSW','201801230LAL','201801230OKC','201801230ORL','201801230SAS','201801240ATL','201801240CHO','201801240DAL','201801240DET','201801240IND','201801240LAC','201801240MEM','201801240PHI','201801240POR','201801250DEN','201801250GSW','201801250MIA','201801250OKC','201801260CHI','201801260CHO','201801260CLE','201801260DAL','201801260MEM','201801260MIL','201801260NOP','201801260PHO','201801260SAS','201801260TOR','201801270ATL','201801270DEN','201801270DET','201801270GSW','201801270IND','201801270MIA','201801270MIN','201801280CHI','201801280CLE','201801280HOU','201801280NOP','201801280OKC','201801280SAS','201801280TOR','201801290ATL','201801290DAL','201801290DEN','201801290IND','201801290MEM','201801290MIL','201801300DET','201801300HOU','201801300LAC','201801300NOP','201801300NYK','201801300SAS','201801300TOR','201801300UTA','201801300WAS','201801310ATL','201801310BOS','201801310BRK','201801310CLE','201801310IND','201801310ORL','201801310PHO','201801310POR','201802010DEN','201802010DET','201802010MIN','201802010SAS','201802010WAS','201802020BOS','201802020BRK','201802020CHO','201802020MIL','201802020OKC','201802020PHI','201802020PHO','201802020SAC','201802020TOR','201802030CLE','201802030DEN','201802030DET','201802030IND','201802030LAC','201802030MIN','201802030ORL','201802030SAC','201802030SAS','201802040BOS','201802040BRK','201802040NYK','201802040OKC','201802040PHO','201802040TOR','201802050DEN','201802050DET','201802050IND','201802050LAC','201802050MIA','201802050NOP','201802050SAC','201802060ATL','201802060BRK','201802060GSW','201802060LAL','201802060NYK','201802060ORL','201802060PHI','201802060TOR','201802070CLE','201802070DET','201802070MEM','201802070MIA','201802070PHO','201802080GSW','201802080LAL','201802080ORL','201802080POR','201802080TOR','201802080WAS','201802090ATL','201802090BOS','201802090CHI','201802090DET','201802090HOU','201802090MIA','201802090PHI','201802090SAC','201802090UTA','201802100BRK','201802100CHI','201802100DAL','201802100GSW','201802100ORL','201802100PHI','201802100PHO','201802110ATL','201802110BOS','201802110CHO','201802110HOU','201802110IND','201802110MIN','201802110OKC','201802110POR','201802120BRK','201802120CHI','201802120DET','201802120GSW','201802120PHI','201802120UTA','201802130DAL','201802130DEN','201802130MIL','201802130MIN','201802130OKC','201802130TOR','201802140BOS','201802140BRK','201802140CHI','201802140DET','201802140HOU','201802140MEM','201802140NOP','201802140NYK','201802140ORL','201802140PHI','201802140POR','201802140UTA','201802150MIL','201802150MIN','201802220CHI','201802220CHO','201802220CLE','201802220GSW','201802220ORL','201802220SAC','201802230DEN','201802230DET','201802230HOU','201802230IND','201802230LAL','201802230MEM','201802230NOP','201802230PHO','201802230TOR','201802230UTA','201802230WAS','201802240GSW','201802240MIA','201802240MIN','201802240NYK','201802240PHI','201802240PHO','201802240SAC','201802240UTA','201802250CHO','201802250CLE','201802250DEN','201802250MIL','201802250WAS','201802260ATL','201802260BOS','201802260BRK','201802260DAL','201802260NOP','201802260NYK','201802260OKC','201802260SAC','201802260TOR','201802260UTA','201802270CHO','201802270CLE','201802270DEN','201802270MIA','201802270MIL','201802270POR','201802280ATL','201802280BOS','201802280DAL','201802280DET','201802280LAC','201802280MEM','201802280ORL','201802280SAS','201802280WAS','201803010CLE','201803010MIA','201803010POR','201803010SAC','201803020ATL','201803020CHI','201803020LAC','201803020MEM','201803020MIL','201803020ORL','201803020PHI','201803020PHO','201803020UTA','201803020WAS','201803030CLE','201803030HOU','201803030MIA','201803030ORL','201803030POR','201803030SAC','201803030SAS','201803040ATL','201803040DAL','201803040LAC','201803040MIL','201803040SAC','201803040TOR','201803040WAS','201803050CHI','201803050CLE','201803050IND','201803050LAL','201803050MIA','201803050SAS','201803050UTA','201803060CHO','201803060DAL','201803060GSW','201803060LAC','201803060OKC','201803060POR','201803060TOR','201803060WAS','201803070CHI','201803070DEN','201803070DET','201803070IND','201803070LAL','201803070MIL','201803070SAC','201803080CHO','201803080GSW','201803080MIA','201803080MIN','201803080OKC','201803090DEN','201803090DET','201803090IND','201803090LAC','201803090MEM','201803090MIL','201803090NOP','201803090POR','201803090SAC','201803090TOR','201803100CHO','201803100DAL','201803100LAC','201803100MIA','201803100OKC','201803110ATL','201803110BOS','201803110BRK','201803110DAL','201803110DEN','201803110LAL','201803110MIN','201803110NOP','201803110NYK','201803120HOU','201803120MEM','201803120OKC','201803120POR','201803130ATL','201803130BRK','201803130CHI','201803130LAL','201803130NOP','201803130NYK','201803130PHI','201803130PHO','201803130SAS','201803130UTA','201803130WAS','201803140BOS','201803140GSW','201803140ORL','201803140SAC','201803150ATL','201803150DEN','201803150HOU','201803150IND','201803150MEM','201803150NYK','201803150POR','201803150SAS','201803150UTA','201803160GSW','201803160LAL','201803160OKC','201803160ORL','201803160PHI','201803160TOR','201803170BRK','201803170CHI','201803170MEM','201803170MIL','201803170NOP','201803170NYK','201803170PHO','201803170POR','201803170SAS','201803170UTA','201803170WAS','201803180LAC','201803180MIN','201803180NOP','201803180TOR','201803190BRK','201803190CLE','201803190IND','201803190MIA','201803190NYK','201803190PHI','201803190SAC','201803190SAS','201803200BOS','201803200MIN','201803200NOP','201803200ORL','201803200PHO','201803200POR','201803200UTA','201803210BRK','201803210CHI','201803210CLE','201803210MIA','201803210MIL','201803210NOP','201803210PHI','201803210SAS','201803220CHO','201803220DAL','201803220HOU','201803220NOP','201803220ORL','201803220SAC','201803230CHI','201803230CLE','201803230GSW','201803230IND','201803230NYK','201803230OKC','201803230POR','201803230SAS','201803230TOR','201803230WAS','201803240DAL','201803240DET','201803240HOU','201803240MEM','201803240ORL','201803240PHI','201803250BRK','201803250GSW','201803250HOU','201803250IND','201803250MIL','201803250OKC','201803250SAC','201803250TOR','201803250WAS','201803260CHO','201803260DET','201803260MIN','201803260PHI','201803260PHO','201803270GSW','201803270HOU','201803270LAC','201803270MIA','201803270NOP','201803270SAC','201803270TOR','201803270WAS','201803280CHO','201803280LAL','201803280MEM','201803280MIN','201803280ORL','201803280PHI','201803280PHO','201803280UTA','201803290DET','201803290GSW','201803290MIA','201803290SAC','201803290SAS','201803300ATL','201803300CLE','201803300DAL','201803300HOU','201803300LAL','201803300OKC','201803300ORL','201803300POR','201803300UTA','201803310BOS','201803310MIA','201803310NYK','201803310SAC','201803310WAS','201804010ATL','201804010BRK','201804010CHI','201804010CHO','201804010CLE','201804010DEN','201804010GSW','201804010LAC','201804010LAL','201804010MIN','201804010NOP','201804010POR','201804010SAS','201804030CHI','201804030CLE','201804030DAL','201804030DEN','201804030HOU','201804030LAC','201804030MIA','201804030MIL','201804030NYK','201804030OKC','201804030PHI','201804030PHO','201804030UTA','201804040ATL','201804040DET','201804040LAL','201804040NOP','201804040ORL','201804040TOR','201804050CLE','201804050DEN','201804050HOU','201804050IND','201804050MIL','201804050UTA','201804060BOS','201804060DET','201804060LAL','201804060MEM','201804060NYK','201804060ORL','201804060PHI','201804060PHO','201804060TOR','201804060WAS','201804070CHI','201804070GSW','201804070HOU','201804070LAC','201804070NYK','201804070SAS','201804080BOS','201804080CHO','201804080LAL','201804080MEM','201804080PHI','201804080PHO','201804080TOR','201804090BRK','201804090DEN','201804090DET','201804090LAC','201804090MIA','201804090MIL','201804090MIN','201804090NYK','201804090SAS','201804100ATL','201804100DAL','201804100IND','201804100LAL','201804100UTA','201804100WAS','201804110BOS','201804110CHI','201804110CLE','201804110LAC','201804110MIA','201804110MIN','201804110NOP','201804110OKC','201804110ORL','201804110PHI','201804110POR','201804110SAC','201804140GSW','201804140PHI','201804140POR','201804140TOR','201804150BOS','201804150CLE','201804150HOU','201804150OKC','201804160GSW','201804160PHI','201804170BOS','201804170POR','201804170TOR','201804180CLE','201804180HOU','201804180OKC','201804190MIA','201804190NOP','201804190SAS','201804200IND','201804200MIL','201804200WAS','201804210MIA','201804210MIN','201804210NOP','201804210UTA','201804220IND','201804220MIL','201804220SAS','201804220WAS','201804230MIN','201804230UTA','201804240BOS','201804240GSW','201804240PHI','201804250CLE','201804250HOU','201804250OKC','201804250TOR','201804260MIL','201804270IND','201804270UTA','201804270WAS','201804280BOS','201804280GSW','201804290CLE','201804290HOU','201804300BOS','201805010GSW','201805010TOR','201805020HOU','201805030BOS','201805030TOR','201805040NOP','201805040UTA','201805050CLE','201805050PHI','201805060NOP','201805060UTA','201805070CLE','201805070PHI','201805080GSW','201805080HOU','201805090BOS','201805130BOS','201805140HOU','201805150BOS','201805160HOU','201805190CLE','201805200GSW','201805210CLE','201805220GSW','201805230BOS','201805240HOU','201805250CLE','201805260GSW','201805270BOS','201805280HOU','201805310GSW']
counter=0
master=[]
lol=[]
print("length: {}".format(len(full_list)))
for i in range(len(full_list)):
    counter+=1
    lol.append(full_list[i])
    if counter>=20:
        master.append(lol)
        lol=[]
        counter=0
print("Done")

In [30]:
for i in range(len(master)):
    scrape_to_save(master[i],i)
    print("finished iteration {}".format(i))

Attempting with 201710170CLE at 2018-06-03 22:24:46.664607
Attempting with 201710170GSW at 2018-06-03 22:24:51.133047
Attempting with 201710180BOS at 2018-06-03 22:24:55.558084
Attempting with 201710180DAL at 2018-06-03 22:24:59.996721
Attempting with 201710180DET at 2018-06-03 22:25:04.458544
Attempting with 201710180IND at 2018-06-03 22:25:08.923372
Attempting with 201710180MEM at 2018-06-03 22:25:13.362203
Attempting with 201710180ORL at 2018-06-03 22:25:17.797240
Attempting with 201710180PHO at 2018-06-03 22:25:22.268271
Attempting with 201710180SAC at 2018-06-03 22:25:26.737703
Attempting with 201710180SAS at 2018-06-03 22:25:31.174934
Attempting with 201710180UTA at 2018-06-03 22:25:35.597967
Attempting with 201710180WAS at 2018-06-03 22:25:40.015999
Attempting with 201710190LAL at 2018-06-03 22:25:44.544227
Attempting with 201710190OKC at 2018-06-03 22:25:49.014659
Attempting with 201710190TOR at 2018-06-03 22:25:53.382102
Attempting with 201710200BRK at 2018-06-03 22:25:57.7389

Attempting with 201711050CLE at 2018-06-03 22:34:44.061594
Attempting with 201711050HOU at 2018-06-03 22:34:48.678923
Attempting with 201711050LAC at 2018-06-03 22:34:53.242184
Attempting with 201711050LAL at 2018-06-03 22:35:00.188020
Attempting with 201711050MIN at 2018-06-03 22:35:04.582077


Finished set
[]
finished iteration 6
Attempting with 201711050NYK at 2018-06-03 22:35:09.211574
Attempting with 201711050ORL at 2018-06-03 22:35:13.573310
Attempting with 201711050POR at 2018-06-03 22:35:42.440283
Attempting with 201711050SAS at 2018-06-03 22:35:46.905912
Attempting with 201711050TOR at 2018-06-03 22:35:51.416144
Attempting with 201711060ATL at 2018-06-03 22:35:55.893576
Attempting with 201711060GSW at 2018-06-03 22:36:00.308203
Attempting with 201711060PHO at 2018-06-03 22:36:04.716622
Attempting with 201711070CLE at 2018-06-03 22:36:09.102450
Attempting with 201711070DEN at 2018-06-03 22:36:13.511486
Attempting with 201711070IND at 2018-06-03 22:36:17.948317
Attempting with 2

KeyboardInterrupt: 

In [7]:
df=pd.read_excel(r"A:\Programming files\Python scripts\Data_Analysis\regular_season_nba_games.xlsx")
win_loss_short(third_performance_solo(df,"GSW"))

In [45]:
del new_standings
new_standings=pd.DataFrame(columns=["Team","Regular_Season","Simulated_Season","3rd_quarter_points","3rd_quarter_performance","3rd_Ratio","3rd_only","Half_time","Regular_Season_Second"])
index=0
for i in team_dictionary:
    #print(team_dictionary[i])
    #create new simulated season
    simulated_season=season_wo_third(df,team_dictionary[i])
    half_time_season=half_time_conversion(df,team_dictionary[i])
    new_standings.loc[index,'Team']=team_dictionary[i]
    new_standings.loc[index,'Regular_Season']=win_loss(simulated_season)[0]
    new_standings.loc[index,'Simulated_Season']=win_loss(simulated_season)[1]
    new_standings.loc[index,'3rd_quarter_points']=round(third_performance_net(team_dictionary[i]),2)
    new_standings.loc[index,'3rd_quarter_performance']=sum(third_performance(team_dictionary[i]))
    new_standings.loc[index,'3rd_Ratio']=round(third_ratio(team_dictionary[i]),2)
    new_standings.loc[index,'3rd_only']=win_loss_short(third_performance_solo(df,team_dictionary[i]))
    new_standings.loc[index,'Half_time'] = win_loss(half_time_season)[0]
    new_standings.loc[index,'Regular_Season_Second'] = win_loss(half_time_season)[1]
    #calculate 3rd quarter performance
    
    index+=1

season_wo_third: ATL-117-93-111-76
season_wo_third: ATL-91-74-109-74
season_wo_third: ATL-104-80-116-81
season_wo_third: ATL-93-64-104-89
season_wo_third: ATL-86-66-91-63
season_wo_third: ATL-100-81-105-83
season_wo_third: ATL-106-81-117-86
season_wo_third: ATL-109-73-119-85
season_wo_third: ATL-104-79-119-91
season_wo_third: ATL-117-80-115-82
season_wo_third: ATL-107-86-110-82
season_wo_third: ATL-104-76-111-92
season_wo_third: ATL-94-69-113-86
season_wo_third: ATL-105-75-106-73
season_wo_third: ATL-126-98-80-54
season_wo_third: ATL-99-72-110-74
season_wo_third: ATL-85-58-96-72
season_wo_third: ATL-103-82-116-90
season_wo_third: ATL-116-86-104-85
season_wo_third: ATL-78-59-112-80
season_wo_third: ATL-114-91-121-85
season_wo_third: ATL-114-87-102-72
season_wo_third: ATL-90-75-110-79
season_wo_third: ATL-96-69-96-66
season_wo_third: ATL-117-85-110-77
season_wo_third: ATL-107-84-111-86
season_wo_third: ATL-114-85-123-91
season_wo_third: ATL-91-73-105-79
season_wo_third: ATL-94-68-96-64
s

season_wo_third: CLE-102-84-99-66
season_wo_third: CLE-116-85-97-73
season_wo_third: CLE-93-73-114-82
season_wo_third: CLE-119-93-112-88
season_wo_third: CLE-107-86-112-83
season_wo_third: CLE-101-70-123-99
season_wo_third: CLE-95-74-114-84
season_wo_third: CLE-107-83-124-94
season_wo_third: CLE-130-100-122-95
season_wo_third: CLE-115-82-117-80
season_wo_third: CLE-124-95-119-96
season_wo_third: CLE-113-87-117-82
season_wo_third: CLE-111-77-104-82
season_wo_third: CLE-104-81-101-76
season_wo_third: CLE-115-88-107-90
season_wo_third: CLE-105-83-105-80
season_wo_third: CLE-116-88-88-72
season_wo_third: CLE-119-87-109-78
season_wo_third: CLE-100-71-99-74
season_wo_third: CLE-113-80-91-63
season_wo_third: CLE-108-90-97-72
season_wo_third: CLE-121-85-114-91
season_wo_third: CLE-116-89-111-81
season_wo_third: CLE-113-88-91-65
season_wo_third: CLE-101-76-95-74
season_wo_third: CLE-102-79-106-74
season_wo_third: CLE-105-79-98-71
season_wo_third: CLE-123-91-114-85
season_wo_third: CLE-121-90-11

season_wo_third: GSW-121-91-122-96
season_wo_third: GSW-128-91-120-94
season_wo_third: GSW-101-81-111-79
season_wo_third: GSW-133-99-103-81
season_wo_third: GSW-117-87-112-79
season_wo_third: GSW-120-86-117-87
season_wo_third: GSW-107-83-115-85
season_wo_third: GSW-141-108-113-84
season_wo_third: GSW-112-78-92-69
season_wo_third: GSW-127-84-108-87
season_wo_third: GSW-97-72-80-60
season_wo_third: GSW-125-81-101-75
season_wo_third: GSW-135-99-114-93
season_wo_third: GSW-110-78-100-81
season_wo_third: GSW-88-67-92-66
season_wo_third: GSW-124-77-116-101
season_wo_third: GSW-118-84-111-69
season_wo_third: GSW-91-68-108-79
season_wo_third: GSW-143-107-94-81
season_wo_third: GSW-110-79-95-73
season_wo_third: GSW-106-79-110-80
season_wo_third: GSW-109-75-109-79
season_wo_third: GSW-133-109-112-84
season_wo_third: GSW-123-86-95-78
season_wo_third: GSW-125-86-115-93
season_wo_third: GSW-101-75-87-57
season_wo_third: GSW-102-71-98-81
season_wo_third: GSW-111-78-104-76
season_wo_third: GSW-112-80

season_wo_third: LAC-121-92-106-78
season_wo_third: LAC-106-73-98-84
season_wo_third: LAC-113-94-105-83
season_wo_third: LAC-117-84-127-97
season_wo_third: LAC-105-84-121-82
season_wo_third: LAC-108-86-107-77
season_wo_third: LAC-125-86-106-78
season_wo_third: LAC-121-93-115-86
season_wo_third: LAC-126-99-105-82
season_wo_third: LAC-113-80-102-74
season_wo_third: LAC-109-79-104-82
season_wo_third: LAC-113-85-125-97
season_wo_third: LAC-118-85-126-95
season_wo_third: LAC-102-80-113-84
season_wo_third: LAC-109-84-100-74
season_wo_third: LAC-112-83-103-88
season_wo_third: LAC-96-80-104-74
season_wo_third: LAC-113-83-103-74
season_wo_third: LAC-104-84-101-74
season_wo_third: LAC-108-83-95-71
season_wo_third: LAC-98-75-112-96
season_wo_third: LAC-114-90-101-85
season_wo_third: LAC-129-97-119-85
season_wo_third: LAC-127-93-134-102
season_wo_third: LAC-128-112-117-89
season_wo_third: LAC-122-93-120-90
season_wo_third: LAC-92-63-105-83
season_wo_third: LAC-128-93-105-87
season_wo_third: LAC-12

season_wo_third: MIL-110-91-129-86
season_wo_third: MIL-110-81-103-71
season_wo_third: MIL-96-69-109-85
season_wo_third: MIL-110-74-103-80
season_wo_third: MIL-94-61-108-91
season_wo_third: MIL-79-63-97-67
season_wo_third: MIL-104-79-95-74
season_wo_third: MIL-101-79-106-80
season_wo_third: MIL-94-69-116-93
season_wo_third: MIL-109-86-105-79
season_wo_third: MIL-116-93-91-54
season_wo_third: MIL-110-85-96-66
season_wo_third: MIL-107-79-95-73
season_wo_third: MIL-89-67-108-89
season_wo_third: MIL-92-73-90-71
season_wo_third: MIL-109-88-94-63
season_wo_third: MIL-103-77-89-72
season_wo_third: MIL-85-77-91-61
season_wo_third: MIL-111-83-104-84
season_wo_third: MIL-97-64-92-74
season_wo_third: MIL-123-97-134-101
season_wo_third: MIL-110-79-110-79
season_wo_third: MIL-114-95-114-76
season_wo_third: MIL-104-73-107-87
season_wo_third: MIL-87-61-110-78
season_wo_third: MIL-96-75-103-70
season_wo_third: MIL-118-85-110-96
season_wo_third: MIL-89-66-92-63
season_wo_third: MIL-99-69-110-81
season_

season_wo_third: OKC-95-74-100-75
season_wo_third: OKC-92-71-92-83
season_wo_third: OKC-103-81-116-76
season_wo_third: OKC-100-78-95-72
season_wo_third: OKC-94-68-94-64
season_wo_third: OKC-96-67-111-81
season_wo_third: OKC-95-77-94-62
season_wo_third: OKC-107-73-79-56
season_wo_third: OKC-120-102-117-86
season_wo_third: OKC-103-74-89-68
season_wo_third: OKC-112-82-107-76
season_wo_third: OKC-124-88-107-82
season_wo_third: OKC-95-72-97-82
season_wo_third: OKC-113-91-116-89
season_wo_third: OKC-133-100-96-70
season_wo_third: OKC-127-97-117-84
season_wo_third: OKC-100-71-114-86
season_wo_third: OKC-106-80-117-83
season_wo_third: OKC-88-70-104-75
season_wo_third: OKC-101-83-91-74
season_wo_third: OKC-95-64-88-72
season_wo_third: OKC-114-81-90-72
season_wo_third: OKC-148-110-124-90
season_wo_third: OKC-109-82-108-82
season_wo_third: OKC-121-93-112-78
season_wo_third: OKC-121-92-108-84
season_wo_third: OKC-122-87-112-87
season_wo_third: OKC-96-74-102-80
season_wo_third: OKC-124-92-127-95
se

season_wo_third: POR-124-86-76-56
season_wo_third: POR-114-85-96-75
season_wo_third: POR-110-85-113-89
season_wo_third: POR-103-79-93-68
season_wo_third: POR-103-77-104-86
season_wo_third: POR-114-83-107-75
season_wo_third: POR-85-59-99-72
season_wo_third: POR-94-64-94-69
season_wo_third: POR-113-92-110-87
season_wo_third: POR-103-75-99-78
season_wo_third: POR-97-76-98-71
season_wo_third: POR-97-79-101-70
season_wo_third: POR-99-69-82-65
season_wo_third: POR-99-75-94-70
season_wo_third: POR-82-63-86-68
season_wo_third: POR-102-75-90-68
season_wo_third: POR-100-83-92-67
season_wo_third: POR-81-61-101-81
season_wo_third: POR-127-86-125-86
season_wo_third: POR-108-76-105-74
season_wo_third: POR-103-66-91-64
season_wo_third: POR-91-70-103-76
season_wo_third: POR-116-91-123-85
season_wo_third: POR-92-63-106-78
season_wo_third: POR-117-81-124-102
season_wo_third: POR-104-76-111-78
season_wo_third: POR-102-82-95-76
season_wo_third: POR-95-70-88-65
season_wo_third: POR-93-68-91-61
season_wo_th

season_wo_third: TOR-129-109-132-98
season_wo_third: TOR-116-91-112-85
season_wo_third: TOR-106-86-117-90
season_wo_third: TOR-114-90-110-83
season_wo_third: TOR-99-70-110-81
season_wo_third: TOR-106-80-112-86
season_wo_third: TOR-96-72-78-58
season_wo_third: TOR-92-65-73-57
season_wo_third: TOR-112-84-101-77
season_wo_third: TOR-108-80-98-78
season_wo_third: TOR-105-80-105-81
half time conversion: TOR-117-58-100-37
half time conversion: TOR-128-62-94-49
half time conversion: TOR-97-50-101-50
half time conversion: TOR-112-53-117-61
half time conversion: TOR-101-45-92-51
half time conversion: TOR-99-54-85-35
half time conversion: TOR-111-46-129-60
half time conversion: TOR-109-53-100-54
half time conversion: TOR-96-46-107-62
half time conversion: TOR-119-65-114-45
half time conversion: TOR-122-66-118-60
half time conversion: TOR-94-49-95-44
half time conversion: TOR-129-76-113-64
half time conversion: TOR-125-64-116-63
half time conversion: TOR-107-54-84-36
half time conversion: TOR-100

In [61]:
print(win_loss(season_wo_third(df,"WAS")))
print(win_loss(half_time_conversion(df,"WAS")))

('40-36-6', '40-40-2')
('40-36-6', '40-37-5')


In [53]:
new_standings
new_standings[["Team","Regular_Season",'Regular_Season_Second']].head()

,Team,Regular_Season,Regular_Season_Second
0,ATL,24-57-1,29-50-3
1,BRK,27-49-6,32-45-5
2,BOS,53-25-4,43-37-2
3,CHO,34-44-4,38-39-5
4,CHI,25-52-5,30-46-6


In [22]:
writer = pd.ExcelWriter('A:\\Programming files\\Python scripts\\Data_Analysis\\new_standings.xlsx', engine='xlsxwriter')
new_standings.to_excel(writer, index=False, sheet_name='full_df')
writer.save()

In [148]:
def third_performance_net(team):
    #filter for cases where the team is playing
    third_q_df=df.loc[(df['Home_Team'] == team) | (df['Away_Team'] == team)]
    third_q_df=third_q_df.reset_index(drop=True)
    third_quarter_avg=[]
    for i in range(len(third_q_df)):
        #Find out how much the team vs other team scored in the 3rd quarter
        if third_q_df.loc[i,"Home_Team"]==team:
            team=int(third_q_df.loc[i,'Home_Q3'])
            #other=int(third_q_df.loc[i,'Away_Q3'])
        else:
            team=int(third_q_df.loc[i,'Away_Q3'])
            #other=int(third_q_df.loc[i,'Home_Q3'])
        #for each game, take the differential and append it    
        third_quarter_avg.append(team)
    third_quarter_avg=sum(third_quarter_avg)/82
    return third_quarter_avg
third_performance_net("LAC")

26.597560975609756

1.1128537628885338
1.1040587038818277


In [ ]:
#how many games were teams winning/losing before the 3rd vs after?